In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from ast import literal_eval
import logging
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time

In [0]:
from geomatic.overpass import (collect_points_of_interest_in_overpass_geometry,
                               from_poi_collection_to_pandas)
from geomatic.utils import label_isochrones_range_min, OVERPASS_POI_TYPES, POINTS_OF_INTEREST_SCHEMA
from geomatic.geographic_handling import from_polygon_coordinates_to_overpass_geometry
from dku_utils import get_current_project_and_variables, DATAFRAMES_READING_CHUNKSIZE

In [0]:
def build_points_of_interest_dataframe(locations_isochrones_df, points_of_interest_tags, isochrones_range_min):
    isochrones_points_of_interest = {}
    missed_points_of_interest = {}
    start_time = time.time()
    locations_identifiers = list(locations_isochrones_df["location_identifier"])

    for isochrone_type in isochrones_range_min:
        missed_points_of_interest[isochrone_type] = []
        isochrones_points_of_interest[isochrone_type] = []
        focus_isochrones = list(locations_isochrones_df[isochrone_type])
        focus_isochrones = [literal_eval(isochrone) for isochrone in focus_isochrones]
        n_isochrones = len(focus_isochrones)

        for isochrone_index, isochrone_attributes in enumerate(zip(locations_identifiers, focus_isochrones)):
            print("\nCollecting points of interest data for isochrone_type : '{}' (n° {}/{})".format(isochrone_type,
                                                                                                  isochrone_index+1,
                                                                                                   n_isochrones))
            isochrone_location = isochrone_attributes[0]
            isochrone = isochrone_attributes[1]
            isochrone_polygon = isochrone["geometry"]["coordinates"][0]
            isochrone_overpass_geometry = from_polygon_coordinates_to_overpass_geometry(isochrone_polygon)
            querry_additional_logs = {"isochrone_location":isochrone_location, "isochrone_type": isochrone_type}
            points_of_interest_data, response_status_code = collect_points_of_interest_in_overpass_geometry(request_session,
                                                                                                            isochrone_overpass_geometry,
                                                                                                            points_of_interest_tags,
                                                                                                            OVERPASS_POI_TYPES,
                                                                                                            **querry_additional_logs)
            response_status_label = "OK" if response_status_code == 200 else "KO"
            spent_time = time.time() - start_time
            average_collection_time =  spent_time/(isochrone_index + 1.0)
            estimated_time_to_finish = (n_isochrones - (isochrone_index + 1.0)) * average_collection_time
            print("""response_status_code : '{}' ({}) |  isochrone_type : '{}' | """\
                  """isochrone_location : '{}' | spent_time : {} seconds"""\
                  .format(response_status_code,
                          response_status_label,
                          isochrone_type,
                          isochrone_location,
                          spent_time))
            print("Estimated time to finish the collection (with isochrone type = '{}') : {}".format(isochrone_type,
                                                                                                     estimated_time_to_finish))
            isochrones_points_of_interest[isochrone_type].append(points_of_interest_data)

            if response_status_code != 200:
                missed_data = {"isochrone_location":isochrone_location,
                               "isochrone_type":isochrone_type ,
                               "isochrone_overpass_geometry": isochrone_overpass_geometry}
                missed_points_of_interest[isochrone_type].append(missed_data)

    # Output dataframe setup :
    df_points_of_interest = pd.DataFrame(columns=POINTS_OF_INTEREST_SCHEMA)
    locations = list(locations_isochrones_df["location_identifier"])
    for isochrone_type in isochrones_range_min:


        for location, poi_collection in zip(locations, isochrones_points_of_interest[isochrone_type]):
            print("isochrone_location : '{}' | isochrone_type : '{}' | Points of interest extraction"\
                  .format(location, isochrone_type))
            poi_collection = poi_collection["elements"]

            df_tmp = from_poi_collection_to_pandas(poi_collection, location, POINTS_OF_INTEREST_SCHEMA)
            df_tmp["isochrone_type"] = isochrone_type
            df_points_of_interest = df_points_of_interest.append(df_tmp)

    final_schema = ["poi_belonging_location"]+[col for col in df_points_of_interest.columns if col != "poi_belonging_location"]
    df_points_of_interest = df_points_of_interest[final_schema]
    return df_points_of_interest

In [0]:
# Read recipe inputs
locations_isochrones = dataiku.Dataset("locations_isochrones")
locations_isochrones_chunks = locations_isochrones.iter_dataframes(chunksize=DATAFRAMES_READING_CHUNKSIZE)

In [0]:
project, variables = get_current_project_and_variables()
global_variables = variables["standard"]
points_of_interest_tags = global_variables["points_of_interest_tags"]
points_of_interest_all_tags = points_of_interest_tags+global_variables["points_of_interest_enrichments"]
isochrones_range_min = global_variables["isochrones_range_min"]
isochrones_range_min = label_isochrones_range_min(isochrones_range_min)

In [0]:
logging.basicConfig(level=logging.DEBUG)
request_session = requests.Session()
retries = Retry(total=10,
                backoff_factor=1,
                status_forcelist=[502, 503, 504, 429],
                method_whitelist=frozenset(['GET', 'POST']))
request_session.mount('http://', HTTPAdapter(max_retries=retries))

In [0]:
# Write recipe outputs
locations_points_of_interest = dataiku.Dataset("locations_points_of_interest_TEST")

with locations_points_of_interest.get_writer() as writer:
    for chunk_idx, chunk_df in enumerate(locations_isochrones_chunks):
        #TODO : DELETE THE SAMPLING BELOW :
        chunk_df = chunk_df.head(1)
        first_row_in_chunk = chunk_idx*DATAFRAMES_READING_CHUNKSIZE + 1
        last_row_in_chunk = (chunk_idx + 1)*DATAFRAMES_READING_CHUNKSIZE

        print("Managing dataset rows {} to {}".format(first_row_in_chunk, last_row_in_chunk))

        chunk_df_points_of_interest = build_points_of_interest_dataframe(chunk_df,
                                                                         points_of_interest_tags,
                                                                         isochrones_range_min)

        for tag_type in points_of_interest_all_tags:
            chunk_df_points_of_interest[tag_type] = ""
            chunk_df_points_of_interest[tag_type] = chunk_df_points_of_interest["tags"].apply(lambda x: x.get(tag_type))
            if tag_type in points_of_interest_tags:
                chunk_df_points_of_interest["has_{}".format(tag_type)] =\
                chunk_df_points_of_interest[tag_type].apply(lambda x: 1 if x is not None else 0)

        # Write the processed dataframe
        if chunk_idx == 0:
            print("Output schema update ...")
            locations_points_of_interest.write_schema_from_dataframe(chunk_df_points_of_interest)
        print("Writing output data (Resuting from processing of input rows {} to {})".format(first_row_in_chunk, last_row_in_chunk))
        writer.write_dataframe(chunk_df_points_of_interest)